In [ ]:
# Note, the first step will be to produce the master calibration files (bias, dark, flat) for your observing run.

# Note, the second step is then to calibrate your science images using the master calibration files

# Note, the third step is to use your photometry program of choice to run aperture photometry. We don't
# necessarily need photometry for all stars, just the exoplanet host and any stars that you wish to treat
# as constant-brightnes reference stars. Keeping it in instrumental magnitudes is fine.

# After you have these, we'll pick up with this example.

In [ ]:
# First, we'll define the vectors of instrumental magnitudes for each star, and a corresponding vector
# for the times that each was observed. 

import numpy as np
import matplotlib.pyplot as plt
import math as math
np.random.seed(314158)

# First, give each star 21 observations at its instrumental magnitude
star0 = np.zeros(21) - 15.0
star1 = np.zeros(21) - 14.7
star2 = np.zeros(21) - 14.5
star3 = np.zeros(21) - 15.3

# And define a vector describing the observation times
otime  = (np.arange(21)-10.0) * (5.0/(60.0*24.0)) # Observe every 5 minutes, units are in days

# Now, inject a transit into star0
star0[5:16] = star0[5:16] + 0.02 # Make the star 0.01 mags (1%) fainter for the central 50 minutes

# And let's plot the nominally perfect light curve for the planet host star. Notice I didn't build in the
# finite start and end durations, since this is just an example. You'll have those in your data, though.

plt.plot(otime,star0,'ro')
plt.xlabel('Time (days)')
plt.ylabel('Instrumental magnitude (mag)')
plt.ylim(-14.975,-15.005)
plt.show()





In [ ]:
# Note that I now add a variety of noise sources:
# 1) Random noise for each star
# 2) A random image-to-image change in brightness (i.e., from a cloud drifting through) that is the same for all stars
# 3) A quadratic change in brightness over time that is similar for each star, but a little different. This
#     is from looking through a different amount of air (airmass) over time.

Random error-causing line so people wont just run the whole thing


# Add random 0.5% noise to each observation of each star
# This is like a photometric error from photon counting statistics
star0 = star0 + np.random.normal(loc=0.0,scale=0.005,size=21)
star1 = star1 + np.random.normal(loc=0.0,scale=0.005,size=21)
star2 = star2 + np.random.normal(loc=0.0,scale=0.005,size=21)
star3 = star3 + np.random.normal(loc=0.0,scale=0.005,size=21)

# Finally, add random attenuation of anywhere between 0 mags and 0.15 mags to each observation
# This is shared across all stars
ihateatmosphere = np.random.uniform(low=0.0,high=0.10,size=21)
star0 = star0 + ihateatmosphere
star1 = star1 + ihateatmosphere
star2 = star2 + ihateatmosphere
star3 = star3 + ihateatmosphere

# Add a slightly different airmass term to every star's brightness
star0 = star0 + np.arange(21)*(0.05/20.0) # gets 0.05 mags fainter over the observation
star1 = star1 + np.arange(21)*(0.04/20.0) # gets 0.04 mags fainter over the observation
star2 = star2 + np.arange(21)*(0.07/20.0) # gets 0.07 mags fainter over the observation
star3 = star3 + np.arange(21)*(0.06/20.0) # gets 0.06 mags fainter over the observation

# Now, let's plot each of the light curves without doing any of the corrections. This should look messy.

plt.plot(otime,star0,'ro')
plt.errorbar(otime,star0,yerr=0.005,linestyle="none")
plt.xlabel('Time (days)')
plt.ylabel('Instrumental magnitude (mag)')
plt.gca().invert_yaxis()
plt.show()



In [ ]:
EXERCISE FOR YOU
Make another version that shows all four stars on the plot, making the others blue, green, and magenta
What do you see?



In [ ]:
## Now let's find the image-to-image brightness change for each image, and remove that effect for the planet host.


diff1 = star1 - star1[0]
diff2 = star2 - star2[0]
diff3 = star3 - star3[0]

diff = np.zeros(21)
for i0 in range(0,21):
    diff[i0] = np.mean([ diff1[i0] , diff2[i0] , diff3[i0]])

star0 = star0 - diff
star1 = star1 - diff
star2 = star2 - diff
star3 = star3 - diff

# And plot the planet host and the other stars


plt.plot(otime,star0,'ro')
plt.errorbar(otime,star0,yerr=0.005,linestyle="none")
plt.xlabel('Time (days)')
plt.ylabel('Instrumental magnitude (mag)')
plt.gca().invert_yaxis()
plt.show()

plt.plot(otime,star1,'bo')
plt.errorbar(otime,star1,yerr=0.005,linestyle="none")
plt.xlabel('Time (days)')
plt.ylabel('Instrumental magnitude (mag)')
plt.gca().invert_yaxis()
plt.show()


plt.plot(otime,star2,'go')
plt.errorbar(otime,star2,yerr=0.005,linestyle="none")
plt.xlabel('Time (days)')
plt.ylabel('Instrumental magnitude (mag)')
plt.gca().invert_yaxis()
plt.show()

plt.plot(otime,star3,'mo')
plt.errorbar(otime,star3,yerr=0.005,linestyle="none")
plt.xlabel('Time (days)')
plt.ylabel('Instrumental magnitude (mag)')
plt.gca().invert_yaxis()
plt.show()



In [ ]:
# First we'll assume there isn't any long-term change in brightness, and measure the depth of the transit



intransit = star0[5:16]
outoftransit = np.concatenate((star0[0:5],star0[16:21]))
outoftransittime = np.concatenate((otime[0:5],otime[16:21]))

print "Mean brightness outside transit:"
print np.mean(outoftransit) , "+/-" , \
                (np.std(outoftransit)/math.sqrt(len(outoftransit)))
print "Mean brightness inside transit: "
print np.mean(intransit) , "+/-" , \
                (np.std(intransit)/math.sqrt(len(intransit)))

transitdepth = math here
transiterror = more math here

print "Depth: " , transitdepth , "+/-" , transiterror

In [ ]:
# Note that we never removed the small airmass-dependent term! If you manage to find reference stars
# of similar color to your science target, maybe it isn't too big. But nonetheless, why take the risk?
# So we want to fit a linear regression on the points outside transit, then correct all of star0 and
# recalculate the answer

m,b = np.polyfit(outoftransittime,outoftransit,1)
print "Slope of residual brightness variation: " , m , " in units of mag/day"

# Still worth correcting, I think

star0 = star0 - whatgoeshere?

# And redo the statistics from above

intransit = star0[5:16]
outoftransit = np.concatenate((star0[0:5],star0[16:21]))

print "Mean brightness outside transit:"
print np.mean(outoftransit) , "+/-" , (np.std(outoftransit)/math.sqrt(len(outoftransit)))
print "Mean brightness inside transit: "
print np.mean(intransit) , "+/-" , (np.std(intransit)/math.sqrt(len(intransit)))

transitdepth = math here
transiterror = more math here

print "Depth: " , transitdepth , "+/-" , transiterror

# Notice you didn't change the result, because this artificially contrived example is symmetric. The universe
# often isn't, though! The error did drop though, because the out-of-eclipse scatter no longer has a component
# due to one side being higher than the other.

# As a final step, let's plot the transit as it's fully corrected to the best of our ability.

plt.plot(otime,star0,'ro')
plt.errorbar(otime,star0,yerr=0.005,linestyle="none")
plt.xlabel('Time (days)')
plt.ylabel('Instrumental magnitude (mag)')
plt.gca().invert_yaxis()
plt.show()